# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
from math import sqrt, isnan
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from util import metrics
from util.load_data import load_data
from util.evaluation import *

from madras_laftr.models import *
from madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 100
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
test_loop = 1

## Load data

In [5]:
x_train, y_train, a_train = load_data('adult', 'train')
raw_data = (x_train, y_train, a_train)

In [6]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [7]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 113), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [8]:
x_valid, y_valid, a_valid = load_data('adult', 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [9]:
x_test, y_test, a_test = load_data('adult', 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

In [10]:
num_batchs = y_test.shape[0]//batch_size
num_batchs *= batch_size
y_test = y_test[:num_batchs]
a_test = a_test[:num_batchs]

## Result file

In [11]:
header = "model_name", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [12]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y_hat, A_hat = evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(y_test, Y_hat, a_test, A_hat)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.09515931457281113 | 0.395439088344574 | 0.549805760383606 | 58900.46875 | 0.7213196286472148 | 0.609375 | -101.54231598143237
> 2 | 0.08453857898712158 | 0.36099570989608765 | 0.5368999242782593 | 58900.46875 | 0.7899535809018567 | 0.6772629310344828 | -101.54231598143237
> 3 | 0.08241008967161179 | 0.3594989776611328 | 0.5613436102867126 | 58900.46875 | 0.8136604774535809 | 0.6722065649867375 | -101.54231598143237
> 4 | 0.07375820726156235 | 0.3300558924674988 | 0.5439064502716064 | 58900.46875 | 0.8236488726790451 | 0.6715848806366047 | -101.54231598143237
> 5 | 0.0671633630990982 | 0.30994361639022827 | 0.5422675609588623 | 58900.46875 | 0.8307775198938991 | 0.6712947612732095 | -101.54231598143237
> 6 | 0.08053227514028549 | 0.35269731283187866 | 0.5555024147033691 | 58900.46875 | 0.8320208885941645 | 0.6666528514588859 | -101.54231598143237
> 7 | 0.06835276633501053 | 0.3150978088378906

### LAFTR for Eq Odds

In [13]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y_hat, A_hat = evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(y_test, Y_hat, a_test, A_hat)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.04424821212887764 | 0.4444217085838318 | 0.5771663188934326 | 58900.46875 | 0.7147712201591512 | 0.5772546419098143 | -101.54231598143237
> 2 | -0.03884057328104973 | 0.4666588604450226 | 0.5831805467605591 | 58900.46875 | 0.7826177055702918 | 0.6721236737400531 | -101.54231598143237
> 3 | -0.06460712105035782 | 0.36757582426071167 | 0.5613971948623657 | 58900.46875 | 0.8023872679045093 | 0.6712118700265253 | -101.54231598143237
> 4 | -0.03596420958638191 | 0.46888166666030884 | 0.5767742991447449 | 58900.46875 | 0.8209963527851458 | 0.6680620026525199 | -101.54231598143237
> 5 | -0.07006118446588516 | 0.40109020471572876 | 0.6112737655639648 | 58900.46875 | 0.8213693633952255 | 0.6674817639257294 | -101.54231598143237
> 6 | -0.06941968202590942 | 0.35240647196769714 | 0.560665488243103 | 58900.46875 | 0.8275033156498673 | 0.6696369363395225 | -101.54231598143237
> 7 | -0.07739249616861343 

### LAFTR for Eq Opp

In [14]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y_hat, A_hat = evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(y_test, Y_hat, a_test, A_hat)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.03196146339178085 | 0.47212380170822144 | 0.5857645273208618 | 58900.46875 | 0.7146468832891246 | 0.5650281830238727 | -101.54231598143237
> 2 | -0.049787670373916626 | 0.398366242647171 | 0.5753890872001648 | 58900.46875 | 0.7813743368700266 | 0.6726210212201591 | -101.54231598143237
> 3 | -0.06214417517185211 | 0.36323386430740356 | 0.5841909050941467 | 58900.46875 | 0.8033819628647215 | 0.6699270557029178 | -101.54231598143237
> 4 | -0.060507021844387054 | 0.37325620651245117 | 0.5883922576904297 | 58900.46875 | 0.8211206896551724 | 0.6704658488063661 | -101.54231598143237
> 5 | -0.07411875575780869 | 0.38282954692840576 | 0.6463629007339478 | 58900.46875 | 0.8269230769230769 | 0.6634615384615384 | -101.54231598143237
> 6 | -0.0708206295967102 | 0.3602815270423889 | 0.6120882034301758 | 58900.46875 | 0.8268816312997347 | 0.665243700265252 | -101.54231598143237
> 7 | -0.06609877943992615 

## Saving into DF then CSV

In [15]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,LAFTR4DP,0.2,0.830386,0.813836,0.940421,0.966609,0.822027,0.881985,0.893334,4035.0,315.0,186.0,371.0,5886.0,1106.0,944.0,2197.0
1,LAFTR4DP,0.5,0.828590,0.829409,0.954216,0.982695,0.829000,0.886977,0.899087,4031.0,319.0,193.0,364.0,5960.0,1032.0,1034.0,2107.0
2,LAFTR4DP,0.7,0.828059,0.827766,0.949094,0.973515,0.827912,0.884455,0.894915,4035.0,315.0,199.0,358.0,5959.0,1033.0,1039.0,2102.0
3,LAFTR4DP,1.0,0.841489,0.856185,0.948795,0.948251,0.848774,0.891926,0.891686,4054.0,296.0,165.0,392.0,6162.0,830.0,1093.0,2048.0
4,LAFTR4EqOdds,0.2,0.830652,0.840978,0.958662,0.982446,0.835783,0.890078,0.900195,4069.0,281.0,213.0,344.0,6085.0,907.0,1146.0,1995.0
5,LAFTR4EqOdds,0.5,0.830652,0.840978,0.958662,0.982446,0.835783,0.890078,0.900195,4069.0,281.0,213.0,344.0,6085.0,907.0,1146.0,1995.0
6,LAFTR4EqOdds,0.7,0.830652,0.840978,0.958662,0.982446,0.835783,0.890078,0.900195,4069.0,281.0,213.0,344.0,6085.0,907.0,1146.0,1995.0
7,LAFTR4EqOdds,1.0,0.830652,0.840978,0.958662,0.982446,0.835783,0.890078,0.900195,4069.0,281.0,213.0,344.0,6085.0,907.0,1146.0,1995.0
8,LAFTR4EqOpp,0.2,0.824202,0.847709,0.950240,0.962171,0.835790,0.882745,0.887859,3962.0,388.0,163.0,394.0,5937.0,1055.0,1038.0,2103.0
9,LAFTR4EqOpp,0.5,0.824202,0.847709,0.950240,0.962171,0.835790,0.882745,0.887859,3962.0,388.0,163.0,394.0,5937.0,1055.0,1038.0,2103.0


In [16]:
result_df.to_csv('results/test_laftr-100.csv')